## *Visualization of a PFSS magnetic field extrapolation*
***

In this example, a PFSS magnetic field model is computed, and the result is then visualized using SHELVIS.

In [1]:
import numpy as np

In [2]:
import sunpy

In [3]:
import astropy.coordinates
import astropy.units as u

In [4]:
import shelvis

### Load magnetogram & process

In [5]:
import cider.magnetogram
import cider.magnetogram.hmi

In [6]:
magnetogram_file = "../data/hmi.mrdailysynframe_polfil_720s.20220328_120000_TAI.Mr_polfil.fits"

In [7]:
raw_magnetogram = cider.magnetogram.hmi.read_hmi_daily_synframe(magnetogram_file)

In [8]:
import cider.utils.map

In [9]:
# Create an empty map with the requested resolution
uniform_map \
    = cider.utils.map.create_full_sun_plate_carree_map(raw_magnetogram,
                                                       deg_per_pixel=1.0,
                                                       frame=raw_magnetogram.coordinate_frame.name)

In [10]:
remapped_magnetogram = cider.magnetogram.regrid_to_grid_of_map(raw_magnetogram, uniform_map)

In [11]:
balanced_magnetogram = cider.magnetogram.Balance.multiplicative(remapped_magnetogram)

### Compute the PFSS model

In [12]:
import cider.models.pfss

In [13]:
r = np.linspace(1.0, 2.5, 128)*astropy.constants.R_sun

In [14]:
# Instantiate the model
pfss = cider.models.pfss.PotentialFieldSourceSurfaceModel(balanced_magnetogram, r)

In [15]:
# Compute the solution
pfss.compute()

In [16]:
# Compute the magnetic field
magnetic_field = pfss.magnetic_field()

In [17]:
import cider.utils.interpolation

In [18]:
nodal_magnetic_field \
        = cider.utils.interpolation.average_face_staggered_to_nodal(magnetic_field)

### Create a Dataset object for SHELVIS

First, the coordinate frame that the result is plotted in needs to be defined. Here, we use the Stonyhurst frame as an example.

In [19]:
frame_of_plot = sunpy.coordinates.HeliographicStonyhurst(lon=0*u.deg, 
                                                         lat=0*u.deg, 
                                                         obstime=pfss.magnetogram.coordinate_frame.obstime)

Create the SHELVIS dataset. In doing so, the result is transformed to the requested frame

In [20]:
import shelvis.io.cider

In [21]:
dataset = shelvis.io.cider.Dataset(coordinate_frame=frame_of_plot)

In [22]:
dataset.from_field(nodal_magnetic_field, frame=pfss.magnetogram.coordinate_frame)

In [24]:
# Add the vector field
dataset.add_vector(nodal_magnetic_field, 
                   name="magnetic_field", 
                   unit=pfss.magnetogram.unit, 
                   frame=pfss.magnetogram.coordinate_frame)

In [25]:
# Add separately the spherical vector components of the magnetic field
dataset.add_scalar(nodal_magnetic_field.data[0], name="br", unit=pfss.magnetogram.unit)
dataset.add_scalar(nodal_magnetic_field.data[1], name="bt", unit=pfss.magnetogram.unit)
dataset.add_scalar(nodal_magnetic_field.data[2], name="bp", unit=pfss.magnetogram.unit)

### Plot the result

To plot the resulting data, first the visualization widget is created. In addition to the coordinate frame, the unit of length used in defining coordinates in the plot needs to be set

In [26]:
vis = shelvis.Visualization(coordinate_frame=frame_of_plot, unit_of_length=astropy.constants.R_sun)

Next, a spherical slice plot is added to the visualization

In [27]:
from shelvis.widgets.implicit import Sphere

In [28]:
vis.add(shelvis.SlicePlot(
            # The dataset to plot
            dataset,
            # The slice surface, here a sphere
            Sphere(radius=1.2,
                    min=1.001,
                    max=2.491,
                    step=0.01,
                    unit=astropy.constants.R_sun),
            # The variable to plot
            variable="br",
            # The unit to use in the plot
            unit="gauss",
            # A name to identify the plot
            name="Sphere slice (br)"))

In [29]:
vis

Visualization(children=(FigureWidget({
    'data': [{'colorbar': {'len': 0.5, 'orientation': 'v', 'title': {'t…